### 과거 봉 데이터 구하기
참고: [https://wikidocs.net/3684](https://wikidocs.net/3684)

In [25]:
import win32com.client
from datetime import date, datetime
import pandas as pd
#import matplotlib.pyplot as plt

ImportError: DLL load failed: 지정된 프로시저를 찾을 수 없습니다.

In [3]:
instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")

In [4]:
symbol = 'A003540'

* `5` 조회 필드
```
0: 날짜(ulong)
1:시간(long) - hhmm
2:시가(long or float)
3:고가(long or float)
4:저가(long or float)
5:종가(long or float)
6:전일대비(long or float) - 주) 대비부호(37)과 반드시 같이 요청해야 함
8:거래량(ulong or ulonglong) 주) 정밀도 만원 단위
9:거래대금(ulonglong)
10:누적체결매도수량(ulong or ulonglong) - 호가비교방식 누적체결매도수량
11:누적체결매수수량(ulong or ulonglong) - 호가비교방식 누적체결매수수량
 (주) 10, 11 필드는 분,틱 요청일 때만 제공
12:상장주식수(ulonglong)
13:시가총액(ulonglong)
14:외국인주문한도수량(ulong)
15:외국인주문가능수량(ulong)
16:외국인현보유수량(ulong)
17:외국인현보유비율(float)
18:수정주가일자(ulong) - YYYYMMDD
19:수정주가비율(float)
20:기관순매수(long)
21:기관누적순매수(long)
22:등락주선(long)
23:등락비율(float)
24:예탁금(ulonglong)
25:주식회전율(float)
26:거래성립률(float)
37:대비부호(char) - 수신값은 GetHeaderValue 8 대비부호와 동일
```

In [5]:
instStockChart.SetInputValue(0, symbol)
instStockChart.SetInputValue(1, ord('2')) # '1': 기간요청, '2': 개수로 요청
instStockChart.SetInputValue(4, 10) # 요청 데이터 개수: 최근 10일치
instStockChart.SetInputValue(5, 5) # 조회 필드 선택
instStockChart.SetInputValue(6, ord('D')) # 차트구분: D(일), W(주), M(월), m(분), T(틱)
instStockChart.SetInputValue(9, ord('1'))

In [6]:
# 블록킹 방식으로 데이터를 요청함
instStockChart.BlockRequest()
# 서버로부터 데이터를 얻을때는 GetHeaderValue와 GetDataValue 메소드를 사용한다.

0

In [12]:
# 레코드 갯수를 구함
num_data = instStockChart.GetHeaderValue(3)
num_field = instStockChart.GetHeaderValue(1)
print(num_data, num_field)

10 1


In [11]:
for i in range(num_data):
    print(instStockChart.GetDataValue(0, i))

21300
21300
21200
20450
20100
19600
19500
19350
19400
19300


In [16]:
# 시가, 고가, 저가, 종가 거래량을 구하기
instStockChart.SetInputValue(5, (0, 2, 3, 4, 5, 8))
instStockChart.BlockRequest() # 재요청
num_data = instStockChart.GetHeaderValue(3)
num_field = instStockChart.GetHeaderValue(1)

In [22]:
ds = {
    'date': [],
    'open': [],
    'high': [],
    'low': [],
    'close': [],
    'value': [],
}
for i in range(num_data):
    for j in range(num_field):
        print(instStockChart.GetDataValue(j, i), end=" ")
    print("")
    dt_val = instStockChart.GetDataValue(0, i)
    dt = datetime(int(dt_val/10000), int((dt_val%10000)/100), dt_val % 100)
    ds['date'].append(dt)
    ds['open'].append(instStockChart.GetDataValue(1, i))
    ds['high'].append(instStockChart.GetDataValue(2, i))
    ds['low'].append(instStockChart.GetDataValue(3, i))
    ds['close'].append(instStockChart.GetDataValue(4, i))
    ds['value'].append(instStockChart.GetDataValue(5, i))

20210916 21300 21500 21250 21500 5801 
20210915 21250 21500 20950 21300 188349 
20210914 20550 21300 20550 21200 315382 
20210913 20100 20500 20100 20450 212387 
20210910 19650 20150 19550 20100 230870 
20210909 19400 19700 19350 19600 141276 
20210908 19250 19500 19200 19500 153052 
20210907 19400 19400 19200 19350 55682 
20210906 19250 19400 19200 19400 64933 
20210903 19150 19400 19100 19300 141864 


In [23]:
df = pd.DataFrame(ds)
df

,date,open,high,low,close,value
0,2021-09-16,21300,21500,21250,21500,5801
1,2021-09-15,21250,21500,20950,21300,188349
2,2021-09-14,20550,21300,20550,21200,315382
3,2021-09-13,20100,20500,20100,20450,212387
4,2021-09-10,19650,20150,19550,20100,230870
5,2021-09-09,19400,19700,19350,19600,141276
6,2021-09-08,19250,19500,19200,19500,153052
7,2021-09-07,19400,19400,19200,19350,55682
8,2021-09-06,19250,19400,19200,19400,64933
9,2021-09-03,19150,19400,19100,19300,141864


df.set_index(df['date'], inplace=True)